In [ ]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd

In [ ]:
# 간단한 데이터셋 불러오기
# 모두 수치형 데이터이기 때문에 사용하기에 용이하다.
from sklearn.datasets import fetch_california_housing
from sklean.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(Housing.data, housing.target)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.fit(x_valid)
x_test = scaler.fit(x_test)

#### 함수형 API를 만들어서 복잡한 모델 만들기
1. 여러개의 input 다루기

In [ ]:
input_A = keras.layers.Input(shape = [5], name = "wide_input")
input_B = keras.layers.Input(shape = [6], name = "deep_input")
hidden1 = keras.layers.Dense(30, activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30, activation = "relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name = "output")(concat)

Model01 = keras.Model(inputs = [input_A, input_B], outputs = [output])

Model01.compile(
    loss = "mse",
    optimizer = keras.optimizers.SGD(lr = "1e-5")
)

x_train_A , x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:,  2:]
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

History = Model01.fit(
    (x_train_A, x_train_B), y_train, epochs = 20,
    validation_data = ((x_valid_A, x_valid_B), y_valid)
)

MSE_test = Model01.evaluate((x_test_A, x_test_B), y_test)
y_pred = Model01.predict((x_new_A, x_new_B))





2. 여러개의 output 다루기

In [ ]:
input_A = keras.layers.Input(shape = [5], name = "wide_input")
input_B = keras.layers.Input(shape = [6], name = "deep_input")
hidden1 = keras.layers.Dense(30, activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30, activation = "relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name = "main_output")(concat)
aux_output = keras.layers.dense(1, name = "aux_output")(hiddden2)

Model02 = keras.Model(inputs = [input_A, input_B], outputs = [output, aux_output])

# 출력이 여러개일 때는 각각의 출력마다 자신만의 손실함수가 필요하다.
# 따라서 모델을 compile할 때에는 손실의 list를 전달해야 한다.
# Keras를 사용하게 되면 이는 나열된 손실을 모두 더하여 최종 손실을 구해서 훈련에 사용하기 때문에 보조 출력을 규제의 목적으로만 사용하고 싶다면 주 출력에 더 많은 가중치를 부여해야 한다.
# Keras 사용자 정의 모델을 compile할때에 loss_weights() 매개변수를 이용해서 손실 가중치를 저장할 수 있다.

3. 가중치가 필요없는 사용자 정의 층 만들기

In [ ]:
# 입력값에 지수 함수를 적용하는 간단한 사용자 정의 layer
MyLayer = keras.layers.Lambda(lambda x: tf.exp(x))

4. 가중치가 필요한 사용자 정의 층 만들기

In [ ]:
class MyDense(keras.layers.Layer):
  def __init__(self, units, activation = None, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)
  
  def build(self, batch_input_shape):
    self.kernel = self.add_weight(
        name = "kernel", shape = [batch_input_shape[-1], self.units],
        initializer = "glorot_normal"
    )
    self.bias = self.add_weight(
        name = "bias", shape = [self.units], initializer = "zeros"
    )
    super().build(batch_input_shape)
  
  def call(self, x):
    return self.activation(x @ self.kernel + self.bias)
  
  def compute_output_shape(self, batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
  
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "units" : self.units, 
            "activation" : keras.activations.serialize(self.activation)}

### 실제 사용할 모델 구현 연습
#### 1. RNN모델
- 학습 시킬 때에 Label_df만을 이용해도 무방하다.
- 단, 중요한 것은 17개마다 다른 사람의 데이터이기 떄문에 연속된 데이터라고 보기는 힘들다.
- 아